<a href="https://colab.research.google.com/github/yt-oh96/Lab_tutorial/blob/master/Task1_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
ori_image = cv2.imread('image1.tif',1)

plt.subplot(121)
plt.imshow(ori_image, cmap='gray')
plt.title('ori_image')

plt.show()


In [ ]:
#3
M = np.float32([[1,0,14],[0,1,9]])
translate_image = cv2.warpAffine(ori_image, M, ori_image.shape[:2])

plt.subplot(121)
plt.imshow(ori_image, cmap='gray')
plt.title('ori_image' )

plt.subplot(122)
plt.imshow(translate_image, cmap='gray')
plt.title('dis_image')

plt.show()

In [ ]:
def SSD(ori, dis):
  err = np.sum((ori - dis) ** 2)
  err /= (ori.shape[0] * ori.shape[1])
 
  return err

In [ ]:
def entropy(image):
  image = np.asarray(image)

  p_image = image / image.sum()

  p_image = p_image[np.nonzero(p_image)]
  return -np.sum(p_image*np.log2(p_image))

def MI(image_A, image_B):
  image_A = np.asarray(image_A.ravel())
  image_B = np.asarray(image_B.ravel())
  print(image_A.shape)

  histogram_A = np.histogram(image_A, 30)[0]
  histogram_B = np.histogram(image_B, 30)[0]
  histogram_AB = np.histogram2d(image_A, image_B, 30)[0]

  H_a = entropy(image_A)
  H_b = entropy(image_B)
  H_ab = entropy(histogram_AB)
  #H_ab = entropy(np.concatenate([image_A, image_B]))
  print(H_a, ' ',H_b,' ', H_ab)
  MI_ab = H_a + H_b - H_ab

  return MI_ab

In [ ]:
#def fmin_optim(ori, dis, mode='', distortion='', x_val=None, y_val=None, lotataion_val=None ):
def fmin_optim_ssd(val):
  ori_image = cv2.imread('image1.tif',1)
  dis_image = cv2.imread('image2.tif',1)

  x,y = ori_image.shape[:2]

  M = np.float32([[1,0, val[0]],[0,1,val[1]]])
  print('x : ',val[0],'y',val[1])
  dst = cv2.warpAffine(ori_image, M, (x,y))

  err = SSD(dst, dis_image)
  print(err)
  global error
  error.append(err)
  return err

def fmin_optim_mi(val):
  ori_image = cv2.imread('image1.tif',1)
  dis_image = cv2.imread('image2.tif',1)

  x,y = ori_image.shape[:2]

  M = np.float32([[1,0, val[0]],[0,1,val[1]]])
  print('x : ',val[0],'y',val[1])
  dst = cv2.warpAffine(ori_image, M, (x,y))

  err = -MI(dst, dis_image)
  print(err)
  
  global error_mi
  error_mi.append(err)
  return err

In [ ]:
def save_step(k):
  global steps
  steps.append(k)

In [ ]:
steps=[]
error=[]
fmin_ssd = scipy.optimize.fmin(fmin_optim_ssd, x0=[-80,90], xtol=0.01 ,ftol = 0.1,full_output=True,callback=save_step, retall=True)

In [ ]:
st = np.array(steps)
print(st.shape)
print(error)
x = []
y = []
for i in range(st.shape[0]):
  x.append(st[i][0])
  y.append(st[i][1])
print(x, y)

In [ ]:
print(fmin_ssd[0])
plt.title('ssd_error, starting potin : -80,90')
plt.plot(error)
plt.show()

In [ ]:
def save_step_mi(k):
  global steps_mi
  steps_mi.append(k)
  
  
steps_mi=[]
error_mi=[]
fmin = scipy.optimize.fmin(fmin_optim_mi, x0=[30,15], xtol=0.01 ,ftol = 0.1,full_output=True,callback=save_step_mi, retall=True)

In [ ]:
st_mi = np.array(steps_mi)
x_mi = []
y_mi = []
for i in range(st_mi.shape[0]):
  x_mi.append(st_mi[i][0])
  y_mi.append(st_mi[i][1])
print(x_mi, y_mi)

print(fmin[0])

In [ ]:
print(fmin[0])
plt.title('mi_error, starting potin : 30,15')
plt.plot(error)
plt.show()

In [ ]:
M = np.float32([[1,0,-14],[0,1,-9]])
dst_image = cv2.warpAffine(translate_image, M, ori_image.shape[:2])

plt.subplot(121)
plt.imshow(ori_image, cmap='gray')
plt.title('ori_image' )

plt.subplot(122)
plt.imshow(dst_image, cmap='gray')
plt.title('dst_image')

plt.show()